In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

Loading Image from Drive and Outputting

In [2]:
img = cv2.imread('dababy.jpg')
height, width, _ = img.shape
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [9]:
def Print(images):
    
    for location in images:
    
        cv2.imshow("Output", images[location]['img'])
        
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()     

In [11]:
def Partition(input, wid = 2, hei = 2):
    
    height, width = input.shape
    location = 0
    images = {}
    
    for cnt_h in range(hei):
        
        for cnt_w in range(wid):
            
            x = width//wid * cnt_w
            y = height//hei * cnt_h
            h = height//hei
            w = width//wid
            
            img2 = input[y:y+h, x:x+w]
            
            NAME = "Image" + str(location)
            cv2.imwrite(NAME + ".png", img2, [cv2.IMWRITE_PNG_COMPRESSION, 0])
            images["Quadrant" + str(location)] = {'img': img2, 'dark' : False}
            location += 1
            
            Print(images)
        
    return images

images = Partition(gray)


In [25]:
def Thresh(img, alpha = 0.5):
    
    height, width = img.shape
    thresh = np.array([255*alpha])
    dark = np.count_nonzero(img < thresh)
    
    return dark/(height*width) > alpha

for location in images:
    
    images[location]['dark'] = Thresh(images[location]['img'])

In [26]:
def Histograms(images):
    
    imgs = []
    
    for location in images:
        
        histogram = cv2.calcHist([images[location]['img']], [0], None, [255], [0, 255])
        
        plt.ylabel('Pixels')
        plt.xlabel('Darkness')
        plt.title('Histogram of Darkness in Image' + location)
        plt.plot(histogram)
        plt.xlim([0,256])
        
        plt.savefig('Graphs/' + location + 'histogram.png')
        imgs.append('Graphs/' + location + 'histogram.png')
        
        plt.clf()
        
    return imgs

Histograms(images)

['Graphs/Quadrant0histogram.png',
 'Graphs/Quadrant1histogram.png',
 'Graphs/Quadrant2histogram.png',
 'Graphs/Quadrant3histogram.png']

<Figure size 432x288 with 0 Axes>

In [27]:
def PowerTransform(image, gamma = 2):
    
    copy = image.copy()
    copy = np.array(255*(copy/255)**gamma, dtype = 'uint8')
    
    return copy

In [28]:
transforms = {}
count = 0

for location in images:
    
    transform = PowerTransform(images[location]['img'])
    NAME = "Quadrant" + str(count)
    
    cv2.imwrite(NAME + ".png", transform, [cv2.IMWRITE_PNG_COMPRESSION, 0])
    transforms[location] = {'img': transform,
                            'dark': Thresh(transform)}
    
    count += 1
    
Histograms(transforms)

['Graphs/Quadrant0histogram.png',
 'Graphs/Quadrant1histogram.png',
 'Graphs/Quadrant2histogram.png',
 'Graphs/Quadrant3histogram.png']

<Figure size 432x288 with 0 Axes>

In [29]:
def Slice(image, bit):
    
    plane = np.full((image.shape[0], image.shape[1]), 2 ** bit, np.uint8)
    res = cv2.bitwise_and(plane, image)
    x = res*255
    
    return x

In [31]:
bitsliced = {}
slices = [2,7]

for slice in slices:
    
    counter = 0
    
    for location in images:
        
        sliced = Slice(images[location]['img'], slice)
        NAME = "Quadrant " + str(slice) + str(count)
        
        cv2.imwrite(NAME + ".png", sliced, [cv2.IMWRITE_PNG_COMPRESSION, 0])
        bitsliced[NAME[16:]] = {'img' : sliced,
                                   'dark': Thresh(sliced)}
        
        count += 1
        
        Print(bitsliced)
        
Histograms(bitsliced)

['Graphs/histogram.png']

<Figure size 432x288 with 0 Axes>

# Video Application

In [32]:
video = cv2.VideoCapture('monkey.mp4')
fps = video.get(cv2.CAP_PROP_FPS)
print(fps)

ret,frame = video.read()
count = 0

58.285


In [ ]:
while ret:
    
    if count % 8 == 0:
        frame = cv2.resize(frame, (400, 400))
        grayvid = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        partitions = Partition(grayvid, wid=2, hei=2)
        
        for location in partitions:
            
            partitions[location]['dark'] = Thresh(partitions[location]['img'], alpha = 0.25)
            
            if partitions[location]['dark']:
                partitions[location]['dark'] = PowerTransform(partitions[location]['img'], gamma = 0.5)
                
            histogramsloc = Histograms(partitions)
            histograms = []
            
            for hist in histogramsloc:
                
                histograms.append(cv2.imread(hist))
                
            HistoTop = np.concatenate((histograms[0], histograms[1]), axis = 1)
            HistoBottom = np.concatenate((histograms[2], histograms[3]), axis = 1)
            Histogram = np.concatenate((HistoTop, HistoBottom), axis = 0)
            cv2.imshow("Histograms", Histogram)
                
            FrameTop = np.concatenate((partitions['Quadrant0']['img'], partitions['Quadrant1']['img']), axis = 1)
            FrameBottom = np.concatenate((partitions['Quadrant2']['img'], partitions['Quadrant3']['img']), axis = 1)
            Frame = np.concatenate((FrameTop, FrameBottom), axis = 0)
            cv2.imshow("Video", Frame)
                
            count = 0
                
        cv2.waitKey(int(1/fps*1000))
        count += 1
        ret, frame = video.read()
        
    video.release()
    cv2.destroyAllWindows()